# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up



In [2]:
# Do all imports and installs here


import os
import sys
import boto3
import datetime
import pandas as pd

In [3]:
!{sys.executable} -m pip install -U ray
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell"

Requirement already up-to-date: ray in /home/anthelix/anaconda3/envs/psyco/lib/python3.6/site-packages (0.8.5)


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope TODO
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data TODO
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here
!ls -1FSash ./dataset
path = './dataset/'

total 1,2G
510M airports_us.csv
509M GlobalLandTemperaturesByCity.csv
205M WDIData.csv
5,8M airport-codes_csv.csv
248K us-cities-demographics.csv
144K immigration_data_sample.csv
 36K I94_SAS_Labels_Descriptions.SAS
4,0K ./
4,0K ../
4,0K 20-years-us-university-dataset/
4,0K airline-delay-and-cancellation-data-2009-2018/
4,0K eda-for-data-sciene-job-market-in-us/
4,0K education-statistics/
4,0K sas_data/


##### UDACITY DATA

In [6]:
#print("__________________________________________________________________________")
# airport-codes_csv
df_airport_world = pd.read_csv(path+'airport-codes_csv.csv')
# print(df_airport.head(2))
#print("__________________________________________________________________________")
# immigration_data_sample
df_immigration = pd.read_csv(path+'immigration_data_sample.csv')
# print(df_immigration.head(2))
#print("__________________________________________________________________________")
# us-cities-demographics
df_demograph = pd.read_csv(path+'us-cities-demographics.csv', sep=";")
# print(df_demograph.head(2))
#print("__________________________________________________________________________")
# download from kaggle the GlobalLandTemperaturesByCity.csv
df_temperature = pd.read_csv(path+'GlobalLandTemperaturesByCity.csv', sep=",")
#print(df_temperature.head(2))
#print("__________________________________________________________________________")

In [7]:
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [8]:
df_sas=spark.read.parquet("./dataset/sas_data")

In [9]:
nRow, nCol = df_airport_world.shape
print("There are {} rows and {} columns in df_airport_world".format(nRow, nCol))
nRow, nCol = df_immigration.shape
print("There are {} rows and {} columns in df_immigration".format(nRow, nCol))
nRow, nCol = df_demograph.shape
print("There are {} rows and {} columns in df_demograph".format(nRow, nCol))
nRow, nCol = df_temperature.shape
print("There are {} rows and {} columns in df_temperature".format(nRow, nCol))

print("There are {} rows and {} columns in df_sas".format(df_sas.count(), len(df_sas.columns)))

There are 55075 rows and 12 columns in df_airport_world
There are 1000 rows and 29 columns in df_immigration
There are 2891 rows and 12 columns in df_demograph
There are 8599212 rows and 7 columns in df_temperature
There are 3096313 rows and 28 columns in df_sas


##### KAGGLE DATA

In [10]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df_airport_us = pd.read_csv(path+'airports_us.csv', nrows = nRowsRead)
nRow, nCol = df_airport_us.shape
print("There are {} rows and {} columns in df_airport_us".format(nRow, nCol))

There are 1000 rows and 15 columns in df_airport_us


In [11]:
# WDIData.csv Indicators developpement
nRowsRead = 1000 # specify 'None' if want to read whole file
df_indicator_dev = pd.read_csv(path+'WDIData.csv', nrows = nRowsRead)
nRow, nCol = df_indicator_dev.shape
print("There are {} rows and {} columns in df_indicator_dev".format(nRow, nCol))

There are 1000 rows and 64 columns in df_indicator_dev


In [12]:
!ls -tRFh ./dataset/ed*


./dataset/education-statistics:
EdStatsSeries.csv    EdStatsData.csv	 EdStatsCountry-Series.csv
EdStatsFootNote.csv  EdStatsCountry.csv

./dataset/eda-for-data-sciene-job-market-in-us:
data-scientist-job-market-in-the-us/

./dataset/eda-for-data-sciene-job-market-in-us/data-scientist-job-market-in-the-us:
fulltimeRM.csv	 fulltimeSU.csv   fulltimeLA.csv  fulltimeAL.csv
fulltimeSD.csv	 fulltimeBOS.csv  fulltimeMA.csv  fulltimeAT.csv
fulltimeSEA.csv  fulltimeCHI.csv  fulltimeMV.csv  fulltimeBO.csv
fulltimeSF.csv	 fulltimeDC.csv   fulltimeNY.csv  alldata.csv


In [14]:
# /dataset/education-statistics
nRowsRead = 1000 # specify 'None' if want to read whole file
df_country_series = pd.read_csv(path+'education-statistics/EdStatsCountry-Series.csv', nrows = nRowsRead)
nRow, nCol = df_country_series.shape
print("There are {} rows and {} columns in df_country_series".format(nRow, nCol))

nRowsRead = 1000 # specify 'None' if want to read whole file
df_country = pd.read_csv(path+'education-statistics/EdStatsCountry.csv', nrows = nRowsRead)
nRow, nCol = df_country.shape
print("There are {} rows and {} columns in df_country".format(nRow, nCol))

nRowsRead = 1000 # specify 'None' if want to read whole file
df_data = pd.read_csv(path+'education-statistics/EdStatsData.csv', nrows = nRowsRead)
nRow, nCol = df_data.shape
print("There are {} rows and {} columns in df_data".format(nRow, nCol))

nRowsRead = 1000 # specify 'None' if want to read whole file
df_foot_note = pd.read_csv(path+'education-statistics/EdStatsFootNote.csv', nrows = nRowsRead)
nRow, nCol = df_foot_note.shape
print("There are {} rows and {} columns in df_foot_note".format(nRow, nCol))


nRowsRead = 1000 # specify 'None' if want to read whole file
df_series = pd.read_csv(path+'education-statistics/EdStatsSeries.csv', nrows = nRowsRead)
nRow, nCol = df_series.shape
print("There are {} rows and {} columns in df_series".format(nRow, nCol))


There are 613 rows and 4 columns in df_country_series
There are 241 rows and 32 columns in df_country
There are 1000 rows and 70 columns in df_data
There are 1000 rows and 5 columns in df_foot_note
There are 1000 rows and 21 columns in df_series


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.